In [1]:
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score, rand_score
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import confusion_matrix
from munkres import Munkres
from scipy.special import comb
from PIL import Image
import random
from sklearn.neighbors import NearestNeighbors



In [2]:
def make_cost_matrix(c1, c2):
    """
    """
    uc1 = np.unique(c1)
    uc2 = np.unique(c2)
    l1 = uc1.size
    l2 = uc2.size
    assert(l1 == l2 and np.all(uc1 == uc2))

    m = np.ones([l1, l2])
    for i in range(l1):
        it_i = np.nonzero(c1 == uc1[i])[0]
        for j in range(l2):
            it_j = np.nonzero(c2 == uc2[j])[0]
            m_ij = np.intersect1d(it_j, it_i)
            m[i,j] =  -m_ij.size
    return m

def translate_clustering(clt, mapper):
    return np.array([ mapper[i] for i in clt ])



def map_label(pred, gt):
    """entry point"""

    num_labels = len(np.unique(gt))

    # cm = confusion_matrix(gt, pred, labels=range(num_labels)) # gets the confusion matrix

    cost_matrix = make_cost_matrix(pred, gt)

    m = Munkres()
    indexes = m.compute(cost_matrix)
    mapper = { old: new for (old, new) in indexes }


    new_labels = translate_clustering(pred, mapper)
    new_cm = confusion_matrix(gt, new_labels, labels=range(num_labels))
    return new_labels, mapper


In [3]:
def most_vote(nn_class, labels, k):
    cur = 0
    for i in range(nn_class.shape[0]):
        max_num = 0
        for j in range(k):
            if ((nn_class[i, :] == nn_class[i, j]).sum() > max_num) or \
            (k > 1 and (nn_class[i, :] == nn_class[i, j]).sum() == max_num and nn_class[i, j] != labels[i]):
                max_num = (nn_class[i, :] == nn_class[i, j]).sum()
                max_class = nn_class[i, j]
        
        if max_class == labels[i]:
            cur += 1
    return cur


def mean_average_precision(index, label):
    correct = (index == label[:, None]).astype(np.int32)
    precision = np.cumsum(correct, 1) / (np.arange(index.shape[1])[None, :] + 1)
    res = (precision * correct).sum(1) / (correct.sum(1) + 1e-6)
    return res.mean()


In [4]:
all_data = {}
for model_dir in os.listdir("latent_code"):
    all_data[model_dir] = {}
    for s in ["val", "test"]:
        test_dir = os.path.join("latent_code", model_dir, s)

        num_test = len(os.listdir(test_dir)) // 2

        classes = [dir for dir in os.listdir(f"{test_dir}/test_0")]
        classes.sort()
        print(test_dir)
        print()

        data = []
        labels = []
        fnames = []

        idx_num = [int(fname.split("_")[-1]) for fname in os.listdir(f"{test_dir}") if "im" not in fname]
        idx_num.sort()
        num_file = 0
        for i, dir in enumerate(classes):
            _dir = os.path.join(f"{test_dir}/test_0")
            for fname in os.listdir(os.path.join(_dir, dir)):
                num_file += 1


        shuffle_idx = np.arange(num_file)
        np.random.shuffle(shuffle_idx)



        for idx in idx_num:
            data_tmp = []
            label_tmp = []
            fnames_tmp = []
            for i, dir in enumerate(classes):
                _dir = os.path.join(f"{test_dir}/test_{idx}")
                for fname in os.listdir(os.path.join(_dir, dir)):
                    data_tmp.append(np.load(os.path.join(_dir, dir, fname)))
                    label_tmp.append(i)
                    fnames_tmp.append(os.path.join(_dir, dir, fname))

            data.append(np.vstack(data_tmp)[shuffle_idx, :])
            labels.append(np.array(label_tmp)[shuffle_idx])
            fnames.append(np.array(fnames_tmp)[shuffle_idx])
        all_data[model_dir][s] = dict(data=data, labels=labels, fnames=fnames, idx_num=idx_num)



latent_code/cnn_share_attn/val

latent_code/cnn_share_attn/test

latent_code/cnn_split_attn/val

latent_code/cnn_split_attn/test

latent_code/cnn/val

latent_code/cnn/test



In [5]:
for m in all_data.keys():
    for s in all_data[m].keys():
        data = all_data[m][s]["data"]
        labels = all_data[m][s]["labels"]
        fnames = all_data[m][s]["fnames"]
        idx_num = all_data[m][s]["idx_num"]
        show_data = {}
        for idx, num in enumerate(idx_num):
            ks = [1, 3, 5, 7, 10]

            knn = NearestNeighbors(n_neighbors=ks[-1] + 1, algorithm='brute').fit(data[idx])
            distances, indices = knn.kneighbors(data[idx])
            remove_idx = (indices == np.arange(indices.shape[0])[:, np.newaxis])
            indices = indices[np.logical_not(remove_idx)].reshape(indices.shape[0], -1)


            for k in ks:

                index = indices[:, :k]
                nn_class = labels[idx][index]
                acc = []
                
                for c in range(len(classes)):
                    cur = 0
                    mask = (labels[idx] == c)
                    nn_class_msk = nn_class[mask]
                    cur = most_vote(nn_class_msk, labels[idx][mask], k)
                    
                    acc.append(cur / nn_class_msk.shape[0])




                total_acc = most_vote(nn_class, labels[idx], k) / nn_class.shape[0]


                # show_data[f"Top{k}_{num}"] =  acc + [total_acc] 
                show_data[f"Top{k}_{num}"] =  [total_acc] 


        print(f"============================={m}-{s}================================================")
        # df = pd.DataFrame(data=show_data, index=classes + ["total"])
        df = pd.DataFrame(data=show_data, index=["total"])

        with pd.option_context(
                            'display.max_columns', None,
                            ):
            print(df)



=============================cnn_share_attn-val================================================
         Top1_0    Top3_0    Top5_0    Top7_0   Top10_0   Top1_10   Top3_10  \
total  0.349964  0.279923  0.342048  0.345646  0.372511  0.722955  0.720556   

        Top5_10   Top7_10  Top10_10   Top1_20   Top3_20   Top5_20   Top7_20  \
total  0.727273  0.730871  0.727513  0.769729  0.762053  0.769489  0.772367   

       Top10_20   Top1_40   Top3_40   Top5_40   Top7_40  Top10_40   Top1_60  \
total  0.761574  0.760614  0.754378  0.755817  0.761094  0.744303  0.727273   

        Top3_60   Top5_60   Top7_60  Top10_60   Top1_80   Top3_80   Top5_80  \
total  0.712641  0.718158  0.718877  0.712641  0.720796  0.707604  0.712161   

        Top7_80  Top10_80  Top1_100  Top3_100  Top5_100  Top7_100  Top10_100  \
total  0.710482  0.700168  0.712401  0.707844  0.701607  0.703766    0.69609   

       Top1_120  Top3_120  Top5_120  Top7_120  Top10_120  Top1_140  Top3_140  \
total  0.723675  0.710002  

In [6]:


for m in all_data.keys():
    for s in all_data[m].keys():
        data = all_data[m][s]["data"]
        labels = all_data[m][s]["labels"]
        fnames = all_data[m][s]["fnames"]
        idx_num = all_data[m][s]["idx_num"]
        show_data = {}
        for idx, num in enumerate(idx_num):
            
            ks = [1, 3, 5, 7, 10]

            knn = NearestNeighbors(n_neighbors=ks[-1] + 1, algorithm='brute').fit(data[idx])
            distances, indices = knn.kneighbors(data[idx])
            remove_idx = (indices == np.arange(indices.shape[0])[:, np.newaxis])
            indices = indices[np.logical_not(remove_idx)].reshape(indices.shape[0], -1)


            nn_class = labels[idx][indices]
            acc = []
            
            for c in range(len(classes)):
                mask = (labels[idx] == c)
                nn_class_msk = nn_class[mask]
                cur = mean_average_precision(nn_class_msk, labels[idx][mask])
                acc.append(cur)




            total_acc = mean_average_precision(nn_class, labels[idx])


            # show_data[f"mAP_{num}"] =  acc + [total_acc] 
            show_data[f"mAP_{num}"] =  [total_acc] 
                
        print(f"============================={m}-{s}================================================")

        # df = pd.DataFrame(data=show_data, index=classes + ["total"])
        df = pd.DataFrame(data=show_data, index=["total"])

        with pd.option_context(
                            'display.max_columns', None,
                            ):
            print(df)




=============================cnn_share_attn-val================================================
          mAP_0    mAP_10    mAP_20    mAP_40    mAP_60    mAP_80   mAP_100  \
total  0.438644  0.758745  0.794347  0.785209  0.750883  0.749034  0.742541   

       mAP_120  mAP_140   mAP_160   mAP_180   mAP_200   mAP_220  mAP_240  \
total  0.75038  0.74029  0.739723  0.746478  0.741855  0.743439  0.74582   

        mAP_260   mAP_280   mAP_300  
total  0.745097  0.746352  0.745508  
=============================cnn_share_attn-test================================================
          mAP_0    mAP_10    mAP_20    mAP_40    mAP_60    mAP_80   mAP_100  \
total  0.442732  0.773245  0.807204  0.794759  0.765211  0.760808  0.756254   

        mAP_120   mAP_140   mAP_160   mAP_180   mAP_200   mAP_220   mAP_240  \
total  0.763001  0.752644  0.753162  0.761451  0.754167  0.758547  0.759436   

        mAP_260   mAP_280   mAP_300  
total  0.758729  0.762895  0.760188  
=========================

In [7]:
def confusion(actual, pred):

    tp_plus_fp = comb(np.bincount(actual), 2).sum()
    tp_plus_fn = comb(np.bincount(pred), 2).sum()
    A = np.c_[(actual, pred)]
    tp = sum(comb(np.bincount(A[A[:, 0] == i, 1]), 2).sum()
             for i in set(actual))
    fp = tp_plus_fp - tp
    fn = tp_plus_fn - tp
    tn = comb(len(A), 2) - tp - fp - fn
    return tp, tn, fp, fn


for m in all_data.keys():
    for s in all_data[m].keys():
        data = all_data[m][s]["data"]
        labels = all_data[m][s]["labels"]
        fnames = all_data[m][s]["fnames"]
        idx_num = all_data[m][s]["idx_num"]
        show_data = {}
        for idx, num in enumerate(idx_num):
            kmeans = KMeans(n_clusters=len(classes), random_state=0, n_init="auto").fit(data[idx])
            tp, tn, fp, fn = confusion(labels[idx], kmeans.labels_)
            total_prec = tp / (tp + fp)
            total_rec = tp / (tp + fn)
            total_acc = (tp + tn) / (tp + tn + fp + fn)
            total_ari = adjusted_rand_score(labels[idx], kmeans.labels_)
            total_nmi = normalized_mutual_info_score(labels[idx], kmeans.labels_)

            show_data[f"{num}"] = [f"{total_prec : .2f}", f"{total_rec : .2f}", f"{total_acc : .2f}", f"{total_ari : .2f}", f"{total_nmi : .2f}"]









    
        print(f"============================={m}-{s}================================================")
        df = pd.DataFrame(data=show_data, index=["precision", "recall", "accuracy", "adjusted RI", "nmi"])
        with pd.option_context(
                            'display.max_columns', None,
                            ):
            print(df)


=============================cnn_share_attn-val================================================
                 0     10     20     40     60     80    100    120    140  \
precision     0.30   0.53   0.50   0.44   0.48   0.45   0.48   0.46   0.47   
recall        0.19   0.22   0.32   0.37   0.37   0.37   0.35   0.40   0.40   
accuracy      0.88   0.85   0.91   0.92   0.92   0.92   0.91   0.93   0.93   
adjusted RI   0.17   0.24   0.34   0.36   0.37   0.36   0.36   0.39   0.40   
nmi           0.39   0.52   0.60   0.60   0.61   0.59   0.62   0.62   0.63   

               160    180    200    220    240    260    280    300  
precision     0.47   0.45   0.48   0.41   0.41   0.44   0.44   0.44  
recall        0.39   0.42   0.44   0.41   0.35   0.41   0.37   0.41  
accuracy      0.92   0.93   0.93   0.93   0.92   0.93   0.92   0.93  
adjusted RI   0.39   0.40   0.42   0.37   0.34   0.39   0.36   0.39  
nmi           0.63   0.62   0.64   0.61   0.59   0.62   0.59   0.62  
===============

In [8]:
best_model = {"cnn" : 0, "cnn_share_attn" : 0, "cnn_split_attn" : 0}
show_data = {}



for m in all_data.keys():
    for s in all_data[m].keys():
        data = all_data[m][s]["data"]
        labels = all_data[m][s]["labels"]
        fnames = all_data[m][s]["fnames"]
        idx_num = all_data[m][s]["idx_num"]
        for idx, num in enumerate(idx_num):
            if num != best_model[m]:
                continue
            
            ks = [1, 3, 5, 7, 10]

            knn = NearestNeighbors(n_neighbors=ks[-1] + 1, algorithm='brute').fit(data[idx])
            distances, indices = knn.kneighbors(data[idx])
            remove_idx = (indices == np.arange(indices.shape[0])[:, np.newaxis])
            indices = indices[np.logical_not(remove_idx)].reshape(indices.shape[0], -1)

            nn_class = labels[idx][indices]

            mean_ap = mean_average_precision(nn_class, labels[idx])
            

            kmeans = KMeans(n_clusters=len(classes), random_state=0, n_init="auto").fit(data[idx])
            tp, tn, fp, fn = confusion(labels[idx], kmeans.labels_)
            total_ari = adjusted_rand_score(labels[idx], kmeans.labels_)
            total_nmi = normalized_mutual_info_score(labels[idx], kmeans.labels_)



            show_data[f"{m}-{s}"] = [most_vote(nn_class, labels[idx], k) / nn_class.shape[0] for k in ks] + [mean_ap, total_ari, total_nmi] 
                


pd.set_option("display.precision", 2)
df = pd.DataFrame(data=show_data, index=[f"Top_{k}" for k in ks] + ["mean_ap", "total_ari", "total_nmi"])

with pd.option_context(
                    'display.max_columns', None,
                    ):
    print(df)

           cnn_share_attn-val  cnn_share_attn-test  cnn_split_attn-val  \
Top_1                    0.35                 0.34                0.33   
Top_3                    0.38                 0.38                0.36   
Top_5                    0.38                 0.38                0.36   
Top_7                    0.37                 0.38                0.36   
Top_10                   0.37                 0.38                0.36   
mean_ap                  0.44                 0.44                0.43   
total_ari                0.17                 0.15                0.13   
total_nmi                0.39                 0.36                0.34   

           cnn_split_attn-test  cnn-val  cnn-test  
Top_1                     0.35     0.49      0.51  
Top_3                     0.37     0.52      0.54  
Top_5                     0.37     0.52      0.53  
Top_7                     0.37     0.52      0.53  
Top_10                    0.37     0.52      0.53  
mean_ap              

In [9]:
for m in all_data.keys():
    for s in all_data[m].keys():
        for dir in os.listdir(os.path.join("latent_code", m, s)):
            count = sum([len(list(os.listdir(os.path.join("latent_code", m, s, dir, i)))) for i in os.listdir(os.path.join("latent_code", m, s, dir))])
            print(f"{os.path.join(m, s, dir)} : {count}")

cnn_share_attn/val/test_im_220 : 0
cnn_share_attn/val/test_im_80 : 0
cnn_share_attn/val/test_260 : 4169
cnn_share_attn/val/test_80 : 4169
cnn_share_attn/val/test_im_140 : 0
cnn_share_attn/val/test_im_0 : 0
cnn_share_attn/val/test_180 : 4169
cnn_share_attn/val/test_100 : 4169
cnn_share_attn/val/test_300 : 4169
cnn_share_attn/val/test_im_120 : 0
cnn_share_attn/val/test_0 : 4169
cnn_share_attn/val/test_10 : 4169
cnn_share_attn/val/test_160 : 4169
cnn_share_attn/val/test_im_10 : 0
cnn_share_attn/val/test_im_240 : 0
cnn_share_attn/val/test_im_60 : 0
cnn_share_attn/val/test_280 : 4169
cnn_share_attn/val/test_60 : 4169
cnn_share_attn/val/test_200 : 4169
cnn_share_attn/val/test_im_160 : 0
cnn_share_attn/val/test_120 : 4169
cnn_share_attn/val/test_im_300 : 0
cnn_share_attn/val/test_im_200 : 0
cnn_share_attn/val/test_im_20 : 0
cnn_share_attn/val/test_20 : 4169
cnn_share_attn/val/test_im_280 : 0
cnn_share_attn/val/test_240 : 4169
cnn_share_attn/val/test_im_40 : 0
cnn_share_attn/val/test_40 : 4169

In [10]:
# show_data = {}
# for idx in range(num_test):
#     kmeans = KMeans(n_clusters=len(classes), random_state=0, n_init="auto").fit(data[idx])
#     acc = []
#     total_mis = 0
#     clusters = []
#     new_cluster, cluster_map = map_label(kmeans.labels_, labels[idx])
#     # cluster_map = {classes[old] : new for old, new in cluster_map.items()}
#     # print(cluster_map)
#     for i in range(len(classes)):
#         mask = (labels[idx] == i)
#         cluster = new_cluster[mask]
#         cur_acc = (cluster == i).mean()
#         acc.append(f"{cur_acc: .2f}")
#         clusters.append(cluster)
#         # if classes[i] == "Violin_Mode" or i == 0:
#         #     print(kmeans.labels_[mask])

#     cluster_class = classes
#     # cluster_class = [""] * len(classes)
#     # for k,v in cluster_map.items():
#     #     cluster_class[v] += f"/{k}"

#     # for i in range(len(cluster_class)):
#     #     if cluster_class[i] == "":
#     #         cluster_class[i] = "None"


#     show_data[f"mAP_{20 * idx + 20}"] = acc + [(new_cluster == labels[idx]).mean()]

#     # print(cluster_map)


# df = pd.DataFrame(data=show_data, index=classes + ["total"])
# print(df)


In [11]:
# class_idx = 0
# mis_class = 0
# mask = (labels == class_idx)
# fname = np.array(fnames)[mask][clusters[class_idx] == mis_class]
# print(fname.shape)
# random.shuffle(fname)
# data = [np.array(Image.open(fname[i][:-4].replace("test", "test_im"))) for i in range(4)]
# data = np.vstack(data)
# fig = plt.figure(figsize=(30, 30)) 
# fig.add_subplot(1, 2, 1)
# plt.imshow(data)
# plt.title(f"{class_idx}-{mis_class}")




# class_idx = 0
# mis_class = 21
# mask = (labels == class_idx)
# fname = np.array(fnames)[mask][clusters[class_idx] == mis_class]
# print(fname.shape)
# random.shuffle(fname)

# data = [np.array(Image.open(fname[i][:-4].replace("test", "test_im"))) for i in range(4)]
# data = np.vstack(data)
# fig.add_subplot(1, 2, 2)
# plt.imshow(data)
# plt.title(f"{class_idx}-{mis_class}")

In [12]:
# for i, cluster in enumerate(clusters):
#     fig, ax = plt.subplots(figsize=(20, 10))
#     counts, bins, patches = ax.hist(cluster, list(range(len(classes))))
#     ax.set_title(classes[i])
#     ax.set_xlabel("Cluster")
#     ax.set_ylabel("Number")
#     ax.set_xticks(bins + 0.5)
#     ax.set_xticklabels(classes, rotation=60, ha="right")






In [13]:
from sklearn.manifold import TSNE
x_embedded = TSNE(n_components=3, learning_rate='auto',
                  init='random', perplexity=3).fit_transform(data)




AttributeError: 'list' object has no attribute 'shape'

In [ ]:
import matplotlib._color_data as mcd
palette = list(mcd.XKCD_COLORS.values())[::10]



for i in range(len(classes)):
    mask = (labels == i)
    x_show = x_embedded[mask, :][:500, :]
    plt.figure()
    plt.scatter(x_show[:, 0], x_show[:, 1], color=palette[3 * i])
    plt.title(classes[i])


plt.figure()
for i in range(len(classes)):
    mask = (labels == i)
    x_show = x_embedded[mask, :][:50, :]
    
    plt.scatter(x_show[:, 0], x_show[:, 1], color=palette[3 * i])

plt.figure()
for i in range(len(classes)):
    mask = (labels == i)
    x_show = x_embedded[mask, :][:50, :].mean(0, keepdims=True)
    
    plt.scatter(x_show[:, 0], x_show[:, 1], color=palette[3 * i], label=classes[i])
    plt.legend(bbox_to_anchor=(1.1, 1.05))



In [ ]:
x_embedded = TSNE(n_components=3, learning_rate='auto',
                  init='random', perplexity=3).fit_transform(data)




In [ ]:
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objs as go
from plotly.graph_objs import *

# Configure Plotly to be rendered inline in the notebook.
plotly.offline.init_notebook_mode()







for i in range(len(classes)):
    mask = (labels == i)
    x_show = x_embedded[mask, :][:500, :]
    # Configure the trace.
    trace = go.Scatter3d(
        x=x_show[:, 0],  
        y=x_show[:, 1], 
        z=x_show[:, 2], 
        mode='markers',
        marker={
            'size': 10,
            'opacity': 0.8,
        },
    )

    # Configure the layout.
    layout = go.Layout(
        margin={'l': 0, 'r': 0, 'b': 0, 't': 0},
        scene=Scene(
                xaxis=XAxis(title=classes[i]),
                yaxis=YAxis(title=classes[i]),
                zaxis=ZAxis(title=classes[i])
            )
    )

    data = [trace]

    plot_figure = go.Figure(data=data, layout=layout)

    # Render the plot.
    plotly.offline.iplot(plot_figure)





In [ ]:
data = []
for i in range(len(classes)):
    mask = (labels == i)
    x_show = x_embedded[mask, :][:100, :]
    cluster = kmeans.labels_[mask]
    # mask = cluster == 14
    # x_show = x_show[mask]
    # Configure the trace.
    trace = go.Scatter3d(
        x=x_show[:, 0],  
        y=x_show[:, 1], 
        z=x_show[:, 2], 
        mode='markers',
        marker={
            'size': 10,
            'opacity': 0.8,
        },
    )

    # Configure the layout.
    layout = go.Layout(
        margin={'l': 0, 'r': 0, 'b': 0, 't': 0},
        scene=Scene(
                xaxis=XAxis(title=classes[i]),
                yaxis=YAxis(title=classes[i]),
                zaxis=ZAxis(title=classes[i])
            )
    )

    data.append(trace)

plot_figure = go.Figure(data=data, layout=layout)

# Render the plot.
plotly.offline.iplot(plot_figure)

